In [34]:
#import thinkplot
#import thinkstats2
import pandas as pd
import numpy as np
import math
import scipy.stats as ss
import matplotlib.pyplot as plt
import thinkplot 
import thinkstats2
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

In [18]:
print("Good Morning!")

Good Morning!


In [19]:
dfd = pd.read_csv("degrees-that-pay-back.csv")
dfd.head()

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Accounting,"$46,000.00","$77,100.00",67.6,"$42,200.00","$56,100.00","$108,000.00","$152,000.00"
1,Aerospace Engineering,"$57,700.00","$101,000.00",75.0,"$64,300.00","$82,100.00","$127,000.00","$161,000.00"
2,Agriculture,"$42,600.00","$71,900.00",68.8,"$36,300.00","$52,100.00","$96,300.00","$150,000.00"
3,Anthropology,"$36,800.00","$61,500.00",67.1,"$33,800.00","$45,500.00","$89,300.00","$138,000.00"
4,Architecture,"$41,600.00","$76,800.00",84.6,"$50,600.00","$62,200.00","$97,000.00","$136,000.00"


In [21]:
df1=dfd[["Undergraduate Major","Starting Median Salary"]]

In [23]:
dfc=pd.read_csv("salaries-by-college-type.csv")
dfc.head(5)

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00"
1,California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
3,"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00","$114,000.00","$66,800.00","$94,300.00","$143,000.00","$190,000.00"
4,Cooper Union,Engineering,"$62,200.00","$114,000.00",NaN,"$80,200.00","$142,000.00",NaN


In [33]:
dfc1=dfc["School Type"]
dfc1.value_counts()

State           175
Liberal Arts     47
Party            20
Engineering      19
Ivy League        8
Name: School Type, dtype: int64

In [26]:
dfr=pd.read_csv("salaries-by-region.csv")
dfr.sample(10)

,School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
29,University of Colorado - Boulder (UCB),Western,"$47,100.00","$97,600.00","$51,600.00","$69,000.00","$128,000.00","$187,000.00"
2,Harvey Mudd College,California,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
25,"California State University (CSU), Stanislaus",California,"$38,000.00","$71,400.00",NaN,"$50,500.00","$94,100.00",NaN
271,University of Massachusetts (UMass) - Amherst,Northeastern,"$46,600.00","$88,200.00","$43,100.00","$61,300.00","$122,000.00","$168,000.00"
169,University of Oklahoma,Southern,"$44,700.00","$82,900.00","$41,200.00","$60,300.00","$114,000.00","$167,000.00"
107,Wayne State University,Midwestern,"$42,800.00","$76,100.00","$40,100.00","$56,200.00","$101,000.00","$139,000.00"
278,Pennsylvania State University (PSU),Northeastern,"$49,900.00","$85,700.00","$46,300.00","$62,000.00","$117,000.00","$160,000.00"
287,Wellesley College,Northeastern,"$42,800.00","$83,500.00",NaN,"$58,600.00","$125,000.00",NaN
113,South Dakota State University (SDSU),Midwestern,"$41,100.00","$73,500.00","$34,100.00","$49,900.00","$99,400.00","$129,000.00"
285,Smith College,Northeastern,"$44,000.00","$83,900.00","$45,100.00","$59,800.00","$129,000.00","$184,000.00"


In [27]:
dfr.describe()

,School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
count,320,320,320,320,273,320,320,273
unique,320,5,168,204,166,217,130,115
top,Stanford University,Northeastern,"$42,600.00","$72,100.00","$43,300.00","$60,700.00","$117,000.00","$150,000.00"
freq,1,100,6,5,5,5,10,10
